# HOMEWORK 2

* Work on Google Analytics Customer Revenue Prediction (https://www.kaggle.com/c/ga-customer-revenue-prediction/overview)
* https://docs.google.com/document/d/1cz4u465B4Oi86gLZSoHeEVIHH5BzSGnodHDaIUWCh0w/edit

#### FROM THE SPEC ABOVE (Copied here for easy reference)

1. Take a look at the training data. There may be anomalies in the data that you may need to factor in before you start on the other tasks. Make a note of the anomalies that you notice. Clean the data first to handle these issues. Explain what you did to clean the data (in bulleted form). (10 points)

2. Generate a heatmap and two other plots (with a subset of variables) visualizing interesting positive and negative correlations. Explain the reason for your choice for these variables and any interesting results associated with them. (15 points)

3. Cluster the data based on geographic information available with a subset of variables that you find relevant.  Include a visualization plot.  Describe your inferences from the clustering and discuss their significance. (15 points)

4. Define a buying score or probability function for each user, which predicts the likelihood of a user buying a product from the GStore. Rank the ten most likely users as who will buy a product from the store.  Does it seem that you that it produces good results?  Report why or why not. (15 points)

5. Identify at least one external data set which you can integrate into your transaction prediction analysis to make it better. Discuss/analyze the extent to which this data helps with the prediction task. (10 points).

6. Finally, build the best prediction model you can to solve the Kaggle task.  Use any data, ideas, and approach that you like. Submit the results of your best models on Kaggle.  Report the rank, score, number of entries, for your highest rank. Include a snapshot of your best score on the leaderboard as confirmation. (20 points)

7. Do a permutation test to determine whether your model really benefits from each input variable you use.   In particular, one at a time, for each relevant input variable, permute the value of this variable and see how they impact the accuracy of the results.  Run enough permutations per variable to establish a p-value of how good your predictions of log of sum of transactions per user are.  You can use whatever metric you wish to score your model (like mean absolute error).  (15 points)



## Data Fields


* **fullVisitorId**- A unique identifier for each user of the Google Merchandise Store.
* **channelGrouping** - The channel via which the user came to the Store.
* **date** - The date on which the user visited the Store.
* **device** - The specifications for the device used to access the Store.
* **geoNetwork** - This section contains information about the geography of the user.
* **socialEngagementType** - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* **totals** - This section contains aggregate values across the session.
* **trafficSource** - This section contains information about the Traffic Source from which the session originated.
* **visitId** - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* **visitNumber** - The session number for this user. If this is the first session, then this is set to 1.
* **visitStartTime** - The timestamp (expressed as POSIX time).
* **hits** - This row and nested fields are populated for any and all types of hits. Provides a record of all page visits.
* **customDimensions** - This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.
* **totals** - This set of columns mostly includes high-level aggregate data.
    * **totals.transactionRevenue** - The target variable we want to predict


In [1]:
import json
from os import path

import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tqdm

In [3]:
# transform df
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
CSV_PATH = './homework2_data/train_v2.csv'
NEW_CSV_PATH = './homework2_data/train_v2_norm.csv'
CHUNK_SIZE = 100_000
count = 0

if not path.exists(NEW_CSV_PATH):
    csv_chunks = pd.read_csv(
        CSV_PATH,
        chunksize=CHUNK_SIZE,
        converters={column: json.loads for column in JSON_COLUMNS}, 
        dtype={'fullVisitorId': 'str'}, # Important!!
    )

    for df in tqdm(csv_chunks):
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        df.to_csv(NEW_CSV_PATH, mode='a', index=False)
        count += len(df)
    
    print(f'Created a new csv in {NEW_CSV_PATH}')
else:
    print('New csv already exists.')

print(f'Processed {count} records.')

New csv already exists.
Processed 0 records.


In [5]:
csv_chunks = pd.read_csv(
    CSV_PATH,
    chunksize=CHUNK_SIZE,
)

for df in tqdm(csv_chunks):
    df.to_csv('./temp.csv', mode='a', index=False)

13it [11:15, 57.21s/it]

KeyboardInterrupt: 

# 1. Detect and handle anomalies

In [9]:
df = pd.read_csv(NEW_CSV_PATH, dtype={'fullVisitorId': 'str'})

/home/julio/.local/share/virtualenvs/stony-brook-data-science-Ue05y5qG/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,7,8,9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pr

In [15]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,100000.0,4.999950e+04,2.886766e+04,0.0,24999.75,49999.5,7.499925e+04,9.999900e+04
totals.bounces,49854.0,1.000000e+00,0.000000e+00,1.0,1.00,1.0,1.000000e+00,1.000000e+00
totals.hits,100000.0,4.367070e+00,8.596425e+00,1.0,1.00,2.0,4.000000e+00,3.310000e+02
totals.newVisits,76365.0,1.000000e+00,0.000000e+00,1.0,1.00,1.0,1.000000e+00,1.000000e+00
totals.pageviews,99989.0,3.647661e+00,6.147223e+00,1.0,1.00,1.0,4.000000e+00,2.300000e+02
totals.sessionQualityDim,46128.0,3.567681e+00,1.078262e+01,1.0,1.00,1.0,1.000000e+00,9.600000e+01
totals.timeOnSite,50026.0,2.419511e+02,4.561073e+02,1.0,29.00,76.0,2.310000e+02,9.837000e+03
totals.totalTransactionRevenue,997.0,1.368457e+08,3.024466e+08,3400000.0,27990000.00,53570000.0,1.146500e+08,5.501000e+09
totals.transactionRevenue,997.0,1.190694e+08,2.588818e+08,1200000.0,22380000.00,46380000.0,1.084700e+08,5.498000e+09
totals.transactions,1002.0,1.055888e+00,3.531933e-01,1.0,1.00,1.0,1.000000e+00,8.000000e+00


In [14]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,100000,NaN,NaN,NaN,49999.5,28867.7,0,24999.8,49999.5,74999.2,99999
channelGrouping,100017,9,Organic Search,43254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customDimensions,100017,7,"[{'index': '4', 'value': 'North America'}]",43785,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,100017,40,2.01611e+07,4055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fullVisitorId,100017,90098,1957458976293878100,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hits,100017,93603,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
socialEngagementType,100017,2,Not Socially Engaged,100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitId,100017,98116,visitId,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitNumber,100017,240,1,75085,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitStartTime,100017,98105,visitStartTime,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
